In [12]:
from typing import Any
import cv2
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

class AlbumentationsDataset(Dataset):
    def __init__(self, file_paths: list, labels, transform=None): 
        # transform default 값을 가지기 때문에 맨 끝에 정의해주지 않으면 에러가 나옴
        # 여러 경로를 받기 때문에 리스트로 정의
        self.file_paths = file_paths  
        # self.file_lists = os.listdir(file_paths)
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        label = self.labels[index]
        file_path = self.file_paths[index]

        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label
    
    def __len__(self):
        return len(self.file_paths)
    

if __name__ == "__main__":
    # torchvision transform이 아닌 albumentation library 사용
    albumentations_transform = A.Compose([ 
        A.Resize(256, 256),
        A.RandomCrop(224, 224), # 이동하면서 random crop resize 내에서 이동해야 하므로 256 * 256보다는 작게 설정함
        A.HorizontalFlip(), # 좌우 반전
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])

    dataset = AlbumentationsDataset(
        ["./sample_data_01/train/dew/2208.jpg", # -- ./ 현재 230619 폴더 내에서 있는 폴더 경로 확인, ../ 상위 폴더에서 들어갈 경로 
         "./sample_data_01/train/rain/1011.jpg", # os 지정이 아닌 그냥 사진만 불러왔기 때문에 사진 경로까지 지정해줘야 하는 듯
         "./sample_data_01/train/frost/3600.jpg"],
         [0, 1, 2],
        transform=albumentations_transform
    )

    for image, label in dataset:
        print(f"Data of dataset : {image}, {label}")


''' Debug. 
    OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
    >>> !_src.empty() in function 이미지가 없으므로 imread()에서 읽은 이미지 확인, 경로 문제'''

Data of dataset : tensor([[[-0.8335, -0.7993, -0.8335,  ..., -0.0629, -0.0458, -0.0458],
         [-0.8335, -0.7993, -0.8164,  ..., -0.2513, -0.2342, -0.0972],
         [-0.8335, -0.8164, -0.8335,  ..., -0.4054, -0.3883, -0.3198],
         ...,
         [-1.9809, -2.0152, -2.0152,  ..., -1.5357, -1.5528, -1.5357],
         [-1.9467, -2.0152, -1.9980,  ..., -1.5185, -1.5185, -1.5357],
         [-1.9980, -2.0152, -2.0323,  ..., -1.4672, -1.4843, -1.4843]],

        [[ 0.0301,  0.0301,  0.0476,  ...,  1.1331,  1.1331,  1.0805],
         [ 0.0301,  0.0476,  0.0826,  ...,  1.0455,  1.0280,  1.1331],
         [ 0.0476,  0.0476,  0.0301,  ...,  0.9580,  0.9405,  1.0105],
         ...,
         [-1.2129, -1.2129, -1.2479,  ..., -0.4951, -0.5126, -0.5301],
         [-1.2129, -1.2304, -1.2654,  ..., -0.4776, -0.4776, -0.4951],
         [-1.2654, -1.2829, -1.3004,  ..., -0.4251, -0.4251, -0.4251]],

        [[-1.7696, -1.8044, -1.7870,  ..., -1.7870, -1.7870, -1.7870],
         [-1.7696, -1.7870,

" Debug. \n    OpenCV(4.7.0) D:\x07\\opencv-python\\opencv-python\\opencv\\modules\\imgproc\\src\\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'\n    >>> !_src.empty() in function 이미지가 없으므로 imread()에서 읽은 이미지 확인, 경로 문제"